In [6]:
# !pip3 install langchain_community

In [7]:
from langchain_community.llms import Ollama

In [8]:
llm = Ollama(model="llama2")
llm.invoke("The first man on the moon was ...")

'\nThe first man to walk on the moon was Neil Armstrong, who stepped onto the lunar surface on July 20, 1969 during the Apollo 11 mission. He famously declared "That\'s one small step for man, one giant leap for mankind" as he took his first steps on the moon\'s surface. Armstrong was followed by fellow astronaut Edwin "Buzz" Aldrin, who also walked on the moon during the same mission.'

In [9]:
llm.invoke("Can you add an appropriate category next to each of the following expenses. Respond with a list of categories separated by commas. For example, Spotify AB by Adyen - \
Entertainment, Beta Boulders Ams Amsterdam Nld - Sports, etc.: \
Taxi Utrecht, Ministerie van Justitie en Veiligheid, Etos AMSTERDAM NLD, Bistro Bar Amsterdam")

'Of course! Here are the categories for each expense:\n\nTaxi Utrecht - Transportation\nMinisterie van Justitie en Veiligheid - Government\nEtos AMSTERDAM NLD - Groceries\nBistro Bar Amsterdam - Dining\n\nI hope this helps! Let me know if you have any other questions.'

### Read transaction data

In [10]:
# Read the transactions_2022_2023.csv file 
import pandas as pd
df = pd.read_csv("/mnt/f/Progetti/local-llms-analyse/"+"transactions_2022_2023.csv")
df.head()

,Date,Name / Description,Expense/Income,Amount (EUR)
0,2023-12-30,Belastingdienst,Expense,9.96
1,2023-12-30,Tesco Breda,Expense,17.53
2,2023-12-30,Monthly Appartment Rent,Expense,451.00
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46
4,2023-12-29,Selling Paintings,Income,13.63


In [11]:
# Get unique transactions in the Name / Description column
unique_transactions = df["Name / Description"].unique()
len(unique_transactions)

31

In [12]:
unique_transactions[1:10]

array(['Tesco Breda', 'Monthly Appartment Rent',
       'Vishandel Sier Amsterdam', 'Selling Paintings',
       'Spotify Ab By Adyen', 'Tk Maxx Amsterdam Da', 'Consulting',
       'Aidsfonds', 'Tls Bv Inz Ov-Chipkaart'], dtype=object)

### Categorise bank transactions with Llama2

In [13]:
# Get index list
#https://stackoverflow.com/questions/47518609/for-loop-range-and-interval-how-to-include-last-step
def hop(start, stop, step):
    for i in range(start, stop, step):
        yield i
    yield stop

index_list = list(hop(0, len(unique_transactions), 30))
index_list

[0, 30, 31]

In [14]:
def categorize_transactions(transaction_names, llm):
    response = llm.invoke("Can you add an appropriate category to the following expenses. For example: Spotify AB by Adyen - Entertainment, Beta Boulders Ams Amsterdam Nld - Sport, etc.. Categories should be less than 4 words. " + transaction_names)
    response = response.split('\n')

    print(response)

    # Put in dataframe
    categories_df = pd.DataFrame({'Transaction vs category': response})
    categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.split(' - ', expand=True)
    
    return categories_df

In [15]:
# Test out the function
categorize_transactions('ISS Catering Services De Meern, Taxi Utrecht, Etos AMSTERDAM NLD, Bistro Bar Amsterdam',
                        llm)

['Of course! Here are the appropriate categories for each expense:', '', '1. Spotify AB by Adyen - Entertainment', '2. Beta Boulders Ams Amsterdam Nld - Sport', '3. ISS Catering Services De Meern - Food & Beverage', '4. Taxi Utrecht - Transportation', '5. Etos AMSTERDAM NLD - Retail', '6. Bistro Bar Amsterdam - Food & Beverage']


,Transaction vs category,Transaction,Category
0,Of course! Here are the appropriate categories...,Of course! Here are the appropriate categories...,None
1,,,None
2,1. Spotify AB by Adyen - Entertainment,1. Spotify AB by Adyen,Entertainment
3,2. Beta Boulders Ams Amsterdam Nld - Sport,2. Beta Boulders Ams Amsterdam Nld,Sport
4,3. ISS Catering Services De Meern - Food & Bev...,3. ISS Catering Services De Meern,Food & Beverage
5,4. Taxi Utrecht - Transportation,4. Taxi Utrecht,Transportation
6,5. Etos AMSTERDAM NLD - Retail,5. Etos AMSTERDAM NLD,Retail
7,6. Bistro Bar Amsterdam - Food & Beverage,6. Bistro Bar Amsterdam,Food & Beverage


In [16]:
# Intialise the categories_df_all dataframe
categories_df_all = pd.DataFrame()

# Loop through the index_list
for i in range(0, len(index_list)-1):
    transaction_names = unique_transactions[index_list[i]:index_list[i+1]]
    transaction_names = ','.join(transaction_names)

    categories_df = categorize_transactions(transaction_names, llm)
    categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)

['Sure! Here are the categories for each expense:', '', '1. Belastingdienst - Taxes', '2. Tesco Breda - Groceries', '3. Monthly Appartment Rent - Housing', '4. Vishandel Sier Amsterdam - Shopping', '5. Selling Paintings - Art/Hobbies', '6. Spotify AB by Adyen - Entertainment', '7. Tk Maxx Amsterdam DA - Shopping', '8. Consulting - Business/Professional Services', '9. Aidsfonds - Charity', '10. TLS BV Inz Ov-Chipkaart - Transportation', '11. Etos Amsterdam - Groceries', '12. Beta Boulders AMS Amsterdam - Entertainment', '13. Salary - Personal Income', '14. Bouldermuur BV Amsterdam - Business/Professional Services', '15. Birtat Restaurant Amsterdam - Dining', '16. Freelancing - Personal Income', '17. Tikkie - Personal Finance', '18. Blogging - Personal Hobbies', '19. Taxi Utrecht - Transportation', '20. Apple Services - Technology', '21. Amazon Lux - Shopping', '22. Classpass* Monthly - Fitness/Wellness', '23. Audible UK AdblCo/Pymt Gbr - Entertainment', '24. Twitch Lux - Personal Income

In [17]:
categories_df_all

,Transaction vs category,Transaction,Category
0,Sure! Here are the categories for each expense:,Sure! Here are the categories for each expense:,None
1,,,None
2,1. Belastingdienst - Taxes,1. Belastingdienst,Taxes
3,2. Tesco Breda - Groceries,2. Tesco Breda,Groceries
4,3. Monthly Appartment Rent - Housing,3. Monthly Appartment Rent,Housing
5,4. Vishandel Sier Amsterdam - Shopping,4. Vishandel Sier Amsterdam,Shopping
6,5. Selling Paintings - Art/Hobbies,5. Selling Paintings,Art/Hobbies
7,6. Spotify AB by Adyen - Entertainment,6. Spotify AB by Adyen,Entertainment
8,7. Tk Maxx Amsterdam DA - Shopping,7. Tk Maxx Amsterdam DA,Shopping
9,8. Consulting - Business/Professional Services,8. Consulting,Business/Professional Services


In [18]:
# categories_df_all.to_csv("categories_df_all.csv", index=False)

In [19]:
# Get unique categories in categories_df_all
unique_categories = categories_df_all["Category"].unique()
unique_categories

array([None, 'Taxes', 'Groceries', 'Housing', 'Shopping', 'Art/Hobbies',
       'Entertainment', 'Business/Professional Services', 'Charity',
       'Transportation', 'Personal Income', 'Dining', 'Personal Finance',
       'Personal Hobbies', 'Technology', 'Fitness/Wellness',
       'Online Services', 'Financial Services', 'Books/Media', 'Sport',
       'Hobby/Leisure'], dtype=object)

In [20]:
# Drop NA values
categories_df_all = categories_df_all.dropna()

# If category contains "Food", then categorise as "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Food"), 'Category'] = "Food and Drinks"
# If category contains "Clothing", then categorise as "Clothing"
categories_df_all.loc[categories_df_all['Category'].str.contains("Clothing"), 'Category'] = "Clothing"
# If category contains "Services", then categorise as "Services"
categories_df_all.loc[categories_df_all['Category'].str.contains("Services"), 'Category'] = "Services"
# If category contains "Health" or "Wellness", then categorise as "Health and Wellness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Health|Wellness"), 'Category'] = "Health and Wellness"
# If category contains "Sport", then categorise as "Sport
#  and Fitness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Sport"), 'Category'] = "Sport and Fitness"
# If category contains "Travel", then categorise as "Travel"
categories_df_all.loc[categories_df_all['Category'].str.contains("Travel"), 'Category'] = "Travel"

In [23]:
# Remove the numbering eg "1. " from Transaction column
categories_df_all.loc['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')
categories_df_all

/tmp/ipykernel_15836/315138465.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categories_df_all.loc['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')


,Transaction vs category,Transaction,Category
2,1. Belastingdienst - Taxes,1. Belastingdienst,Taxes
3,2. Tesco Breda - Groceries,2. Tesco Breda,Groceries
4,3. Monthly Appartment Rent - Housing,3. Monthly Appartment Rent,Housing
5,4. Vishandel Sier Amsterdam - Shopping,4. Vishandel Sier Amsterdam,Shopping
6,5. Selling Paintings - Art/Hobbies,5. Selling Paintings,Art/Hobbies
7,6. Spotify AB by Adyen - Entertainment,6. Spotify AB by Adyen,Entertainment
8,7. Tk Maxx Amsterdam DA - Shopping,7. Tk Maxx Amsterdam DA,Shopping
9,8. Consulting - Business/Professional Services,8. Consulting,Services
10,9. Aidsfonds - Charity,9. Aidsfonds,Charity
11,10. TLS BV Inz Ov-Chipkaart - Transportation,10. TLS BV Inz Ov-Chipkaart,Transportation


In [24]:
# Merge the categories_df_all with the transactions_2022_2023.csv dataframe (df)
df = pd.read_csv("/mnt/f/Progetti/local-llms-analyse/"+"transactions_2022_2023.csv")
df.loc[df['Name / Description'].str.contains("Spotify"), 'Name / Description'] = "Spotify Ab By Adyen"
df = pd.merge(df, categories_df_all, left_on='Name / Description', right_on='Transaction', how='left')
df

,Date,Name / Description,Expense/Income,Amount (EUR),Transaction vs category,Transaction,Category
0,2023-12-30,Belastingdienst,Expense,9.96,NaN,NaN,NaN
1,2023-12-30,Tesco Breda,Expense,17.53,NaN,NaN,NaN
2,2023-12-30,Monthly Appartment Rent,Expense,451.00,NaN,NaN,NaN
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46,NaN,NaN,NaN
4,2023-12-29,Selling Paintings,Income,13.63,NaN,NaN,NaN
5,2023-12-29,Spotify Ab By Adyen,Expense,12.19,NaN,NaN,NaN
6,2023-12-23,Tk Maxx Amsterdam Da,Expense,27.08,NaN,NaN,NaN
7,2023-12-22,Consulting,Income,541.57,NaN,NaN,NaN
8,2023-12-22,Aidsfonds,Expense,10.70,NaN,NaN,NaN
9,2023-12-20,Consulting,Income,2641.93,NaN,NaN,NaN


In [26]:
df.to_csv("/mnt/f/Progetti/local-llms-analyse/nv_"+"transactions_2022_2023_categorized.csv", index=False)